In [1]:
from crewai import Crew, Agent, Task, LLM
from crewai.tools import tool, BaseTool


In [2]:
@tool
def sum_tool(*args) -> float:
    ''' This tool is used to sum the numbers '''
    sum= 0
    for num in args:
        sum= sum + num
    return sum

In [3]:
from pydantic import Field, BaseModel
from typing import Tuple, Type

class MyToolInput(BaseModel):
    args: Tuple= Field(description= "The tuple containing multiple float(number)")

class sub_tool(BaseTool):
    name: str= 'Sub_tool'
    description: str= "This tool is used to subtract the numbers"
    args_schema: Type[BaseModel]= MyToolInput


    def _run(self, *args) -> float:
        self.result= 0
        for num in args:
            self.result= num - self.result

        return self.result




In [4]:
@tool
def mul_tool (a: float, b: float) -> float:
    "This tool is used for multiplication of two numbers"

    return a*b
        

In [5]:
@tool
def div_tool(a:float, b:float)-> float:
    "This tool is used to divide two numbers where a is numerator and b is denominator"
    return a/b

In [6]:
class InputForModulus(BaseModel):
    num1: float= Field(description= "A float number which is used for modulos i.e dividend")
    num2: float= Field(description= "Another float number used for modulus i.e divisior")


class Modulus(BaseTool):
    name: str= "modulo_tool"
    description: str= "This tool is used for the modulo operation between two numbers"
    args_schema: Type[BaseModel]= InputForModulus


    def _run(self, num1: float, num2):
        return num1%num2

In [7]:
llm= LLM(
    model= 'huggingface/openai/gpt-oss-120b',
    temperature= 0.5
)

'''import os
from dotenv import load_dotenv
load_dotenv()
apikey= os.getenv("google2")
gemini_llm= LLM(
    model= "gemini/gemini-2.0-flash",
    api_key= apikey
   '''


llm2= LLM(
    model= 'huggingface/meta-llama/Llama-3.1-8B-Instruct'
)

In [8]:
calculator_agent= Agent(
    role= "Math Teacher",
    goal= "To perform different calculations on the basis of the given user query: {query}",
    backstory= "You are a genius math Tutorial and can solve mathematical problem as fast as a calculator. You have differet tools for different type of operation", 
    llm= llm2,
    function_calling_llm= llm,
    allow_delegation= False,
    Tools= [sum_tool,mul_tool, div_tool, sub_tool, Modulus]
    
    
)

In [9]:
class structured_output(BaseModel):
    Question: str= Field(description= "The Mathematical Question from the given user query")
    Answer: float= Field("The calculated answer")

In [10]:
task= Task(
    description= "This task is related for performing mathematical calculations on the basis of given user query: {query}",
    expected_output= "The answer of given user mathematical problem: {query}",
    agent= calculator_agent,
    output_pydantic= structured_output,
    
    

)

In [11]:
calculator= Crew(
    agents= [calculator_agent],
    tasks= [task],
    output_log_file= 'log.txt',
    planning= True,
    planning_llm= llm,
    verbose= True,
    tracing= True
)

In [12]:
result= calculator.kickoff({'query': 'Can you find 3+6-7/9%6 followd by 6+6/2'})
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 68056ae4-3610-4853-8f6f-dd55d627f333                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


[2025-12-19 20:24:13][INFO]: Planning the crew execution


c:\Users\Yunish\miniconda3\envs\myenv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Math Teacher                                                                                            │
│                                                                                                                 │
│  Task: This task is related for performing mathematical calculations on the basis of given user query: Can you  │
│  find 3+6-7/9%6 followd by 6+6/21. **Read and Understand the Query**                                            │
│     - Identify the two separate arithmetic expressions: (a) 3+6-7/9%6 and (b) 6+6/2.                            │
│  2. **Recall Operator Precedence Rules**                                                                        │
│     - Parentheses (none present).                                                                               │
│     - Exponents (none).                                                                                         │
│     - Multiplication, Division, Modulo (left‑to‑right).                                                         │
│     - Addition and Subtraction (left‑to‑right).                                                                 │
│  3. **Break Down Expression (a): 3+6-7/9%6**                                                                    │
│     a. Compute the division part: 7 ÷ 9 = 0.777777... (retain sufficient decimal precision).                    │
│     b. Compute the modulo of the division result with 6: 0.777777... % 6 = 0.777777... because the dividend is  │
│  smaller than the divisor.                                                                                      │
│     c. Now handle addition and subtraction left‑to‑right:                                                       │
│        - First addition: 3 + 6 = 9.                                                                             │
│        - Then subtraction: 9 - 0.777777... = 8.222222... (rounded as appropriate, e.g., 8.2222).                │
│  4. **Break Down Expression (b): 6+6/2**                                                                        │
│     a. Compute the division first: 6 ÷ 2 = 3.                                                                   │
│     b. Perform the addition: 6 + 3 = 9.                                                                         │
│  5. **Verify Calculations**                                                                                     │
│     - Re‑evaluate each step mentally or with a simple calculator to ensure no arithmetic mistake.               │
│  6. **Prepare the Final Response**                                                                              │
│     - Clearly present the results in the order asked by the user.                                               │
│     - Example wording: "The result of 3+6-7/9%6 is approximately 8.2222, and the result of 6+6/2 is 9."         │
│  7. **Deliver the Answer**                                                                                      │
│     - Output the final results to the user in a concise, friendly manner, ensuring the numbers are formatted    │
│  consistently (e.g., two decimal places if desired).                                                            │
│  8. **Optional: Explain the Process**                                                                           │
│     - If the user seems interested, briefly describe why the division and modulo were performed before          │
│  addition/subtraction, reinforcing the operator precedence rule.                                                │
│                                                                                                                 │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: adcb9f8f-d386-4ecd-b1f6-42ea28bd1808                                                                     │
│  Agent: Task Execution Planner                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

c:\Users\Yunish\miniconda3\envs\myenv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b6aa3b43-9014-4c14-a0a6-13467d27707c                                                                     │
│  Agent: Math Teacher                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Question='Can you find 3+6-7/9%6 followed by 6+6/2' Answer=9.2222


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 68056ae4-3610-4853-8f6f-dd55d627f333                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "Question": "Can you find 3+6-7/9%6 followed by 6+6/2",                                                      │
│    "Answer": 9.2222                                                                                             │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

┌───────────────────────── Trace Batch Finalization ──────────────────────────┐
│ ✅ Trace batch finalized with session ID:                                   │
│ 211348e8-d7fe-4315-94ff-f36897447fee                                        │
│                                                                             │
│ 🔗 View here:                                                               │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/211348e8-d7fe-43 │
│ 15-94ff-f36897447fee?access_code=TRACE-125b6acc54                           │
│ 🔑 Access Code: TRACE-125b6acc54                                            │
└─────────────────────────────────────────────────────────────────────────────┘
